In [56]:
import tarfile
import pandas as pd
import numpy as np
import os
import requests
import dotenv

In [ ]:
# import census data
#tar = tarfile.open('safegraph_open_census_data_2019.tar.gz')
#tar.extractall()
#tar.close()

In [59]:
dotenv.load_dotenv()
key = os.getenv('census_key')

In [2]:
emsdata = pd.read_csv('locationData.csv')
emsdata['Coordinates']=list(zip(emsdata['IncidentLatitude'],emsdata['IncidentLongitude']))

,Unnamed: 0,Number,IncidentAddress,Street,Town,State,Zip,IncidentLatitude,IncidentLongitude,Coordinates
0,0,2016-00006845,2627 Jefferson Park Charlottesville VA 22903,2627 Jefferson Park,Charlottesville,VA,22903,38.017795,-78.517822,"(38.017795, -78.517822)"
1,4,2017-00000006,206 HARTMANS MILL Charlottesville VA 22902,206 HARTMANS MILL,Charlottesville,VA,22902,38.020247,-78.488447,"(38.020247, -78.488447)"
2,8,2017-00000007,102 14TH Northwest Charlottesville VA 22903,102 14TH Northwest,Charlottesville,VA,22903,38.034384,-78.499341,"(38.034384, -78.499341)"
3,11,2017-00000012,2307 North BERKSHIRE B Charlottesville VA 22901,2307 North BERKSHIRE B,Charlottesville,VA,22901,38.063451,-78.497554,"(38.063451, -78.497554)"
4,16,2017-00000014,1311 CARLTON Charlottesville VA 22902,1311 CARLTON,Charlottesville,VA,22902,38.022725,-78.468439,"(38.022725, -78.468439)"


In [3]:
# set path containing datafiles
data_path = '.\safegraph_open_census_data_2019\data'
metadata_path = '.\safegraph_open_census_data_2019\metadata'

# read in file names
data_files = [f for f in os.listdir(data_path) if 'csv' in f]
metadata_files = [f for f in os.listdir(metadata_path) if 'csv' in f]

# read in dataframes
data_dfs = {(f.split('.')[0]): pd.read_csv(str(data_path + '\\' + f)) for f in data_files}
metadata_dfs = {(f.split('.')[0]): pd.read_csv(str(metadata_path + '\\' + f)) for f in metadata_files}

In [4]:
testdf = data_dfs[data_files[0].split('.')[0]]
testdf

,census_block_group,B01001e1,B01001m1,B01001e2,B01001m2,B01001e3,B01001m3,B01001e4,B01001m4,B01001e5,...,B01002He3,B01002Hm3,B01002Ie1,B01002Im1,B01002Ie2,B01002Im2,B01002Ie3,B01002Im3,B01003e1,B01003m1
0,10010201001,730,222,293,87,10,12,26,25,6,...,37.9,21.1,NaN,NaN,NaN,NaN,NaN,NaN,730,222
1,10010201002,1263,291,614,148,24,29,29,22,58,...,38.6,7.5,NaN,NaN,NaN,NaN,NaN,NaN,1263,291
2,10010202001,835,222,393,137,40,37,69,63,16,...,31.9,24.3,65.6,64.9,NaN,NaN,65.6,64.9,835,222
3,10010202002,1124,166,665,132,39,25,46,39,29,...,40.2,6.1,NaN,NaN,NaN,NaN,NaN,NaN,1124,166
4,10010203001,2774,330,1423,225,20,24,143,84,121,...,40.7,12.9,27.8,0.5,27.5,0.2,28.5,0.4,2774,330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220328,560459511001,1450,276,734,146,76,42,44,45,49,...,45.6,9.2,NaN,NaN,NaN,NaN,NaN,NaN,1450,276
220329,560459511002,1844,277,1116,179,14,21,25,30,64,...,52.3,8.9,33.8,6.1,33.8,6.1,NaN,NaN,1844,277
220330,560459513001,1314,359,712,196,42,40,98,80,0,...,38.3,10.8,NaN,NaN,NaN,NaN,NaN,NaN,1314,359
220331,560459513002,1164,254,553,147,47,46,17,27,0,...,48.2,16.3,NaN,NaN,NaN,NaN,NaN,NaN,1164,254


In [5]:
# pull out metdata
coldata = metadata_dfs[metadata_files[0].split('.')[0]]
statedata = metadata_dfs[metadata_files[1].split('.')[0]]
locdata = metadata_dfs[metadata_files[2].split('.')[0]]
blockcodes = pd.read_csv('GeocodeResults.csv')

In [6]:
# add locdata
testdf_loc = testdf.merge(locdata[['census_block_group','latitude','longitude']])

# group colums as id or values
id_vars = ['census_block_group','latitude','longitude']
val_vars = list(np.setdiff1d(list(testdf_loc.columns),id_vars))

# reshape dataframe and merge with coldata
testdf_cl = pd.melt(testdf_loc, id_vars, val_vars) \
                .rename(columns={'variable':'table_id'}) \
                .merge(coldata)

testdf_cl

,census_block_group,latitude,longitude,table_id,value,table_number,table_title,table_topics,table_universe,field_level_1,field_level_2,field_level_3,field_level_4,field_level_5,field_level_6,field_level_7,field_level_8,field_level_9,field_level_10
0,10010201001,32.465832,-86.489661,B01001e1,730.0,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
1,10010201002,32.485867,-86.489671,B01001e1,1263.0,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
2,10010202001,32.480082,-86.474974,B01001e1,835.0,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
3,10010202002,32.464435,-86.469766,B01001e1,1124.0,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
4,10010203001,32.480175,-86.460792,B01001e1,2774.0,B01001,Sex By Age,Age and Sex,Total population,Estimate,SEX BY AGE,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33931277,560459511001,43.995102,-104.788256,B01003m1,276.0,B01003,Total Population,Population Total,Total population,MarginOfError,TOTAL POPULATION,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
33931278,560459511002,43.768531,-104.478499,B01003m1,277.0,B01003,Total Population,Population Total,Total population,MarginOfError,TOTAL POPULATION,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
33931279,560459513001,43.881603,-104.257527,B01003m1,359.0,B01003,Total Population,Population Total,Total population,MarginOfError,TOTAL POPULATION,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN
33931280,560459513002,43.854118,-104.180737,B01003m1,254.0,B01003,Total Population,Population Total,Total population,MarginOfError,TOTAL POPULATION,Total population,Total,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Specify call and parameters
search_url = 'https://geo.fcc.gov/api/census/area?'

# get coordinates for lookup
coord = emsdata[~((emsdata['IncidentLatitude'].isna())|emsdata['IncidentLongitude'].isna())]['Coordinates'].drop_duplicates()

# fetch data and save as json
blockmap = [requests.get(search_url,params={'lat': la, 'lon': lo}) \
     .json()['results'][0]['block_fips'] \
     for la, lo in coord]

In [12]:
blocks = pd.DataFrame({
    'Coordinates':coord,
    'BlockGroup':blockmap
})

In [14]:
emsdata_bg = emsdata.merge(blocks, left_on='Coordinates', right_on='Coordinates')
emsdata_bg['BlockGroup12'] = emsdata_bg['BlockGroup'].str[:12]
emsdata_bg.to_csv('emsmerged.csv')

In [104]:
emsdata_bg['BlockGroup12'][0]

'515400005023'

In [99]:
va = pd.read_csv('./virginia/g20195va.csv')

census resources:
- example blockgroup: https://api.census.gov/data/2019/pdb/blockgroup/examples.html
- api document: https://www.census.gov/content/dam/Census/data/developers/api-user-guide/api-guide.pdf
- handbook: https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_api_handbook_2020_ch02.pdf
- variables: https://api.census.gov/data/2019/acs/acs5/variables.html
- data surveys: https://api.census.gov/data.html
- query exmaples: https://www.census.gov/data/developers/guidance/api-user-guide/query-examples.html
- geoindentifiers: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html
- tutorial: https://github.com/datamade/census
- api intro: https://www.census.gov/data/developers/data-sets/acs-5year.html

In [119]:
from census import Census
#from us import states

c = Census(key)

#block group level income data
c.acs5.get(('NAME', 'B19013_001E'),
          {'for': 'block group:1',
          'in':'state:51 county:540 tract:000502'})[0]['B19013_001E']

117150.0